In [1]:
import pandas as pd 
from IPython.display import display, Markdown

with open('browser_history_file_path.txt', 'r') as file:
    file_path = file.read()

df = pd.read_csv(file_path)

# convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])
max_date = df['Date'].max()
min_date = df['Date'].min()

# using markdown to display the start and end date of the available browsing history
display(Markdown(f' ### Earliest date:  __{min_date}__ \n ### Latest date:  __{max_date}__'))

 ### Earliest date:  __2023-01-30 20:31:28__ 
 ### Latest date:  __2024-09-18 13:25:32__

Choose a start and end date for the desired browsing history to analyze.

Bear in mind that the dataframe cannot exceed approximately 800,000 characters, so the date range should be chosen accordingly. It might be a trial and error process to find the largest possible date range that fits within the character limit.

In [2]:
start = '2023-10-10 09:00:00'
end =   '2023-10-10 17:30:00'

# Filter the DataFrame based on the date range
filtered_df = df[(df['Date'] >= start) & (df['Date'] <= end)]

# Select the desired columns
result_df = filtered_df[['Date', 'URL']]
result_df

,Date,URL
13226,2023-10-10 10:02:20,https://www.pythonanywhere.com/
13227,2023-10-10 10:02:20,https://www.pythonanywhere.com/
13228,2023-10-10 10:02:51,https://www.pythonanywhere.com/pricing/
13229,2023-10-10 10:02:51,https://www.pythonanywhere.com/pricing/
13230,2023-10-10 10:03:25,https://www.pythonanywhere.com/registration/re...
...,...,...
13395,2023-10-10 17:27:24,https://yosefc.pythonanywhere.com/?year=5774
13396,2023-10-10 17:29:53,https://www.pythonanywhere.com/user/yosefc/con...
13397,2023-10-10 17:29:53,https://www.pythonanywhere.com/user/yosefc/con...
13398,2023-10-10 17:30:00,https://www.pythonanywhere.com/user/yosefc/con...


__For the amount of data we are feeding the model, we can only use the gpt-4 mini model. The other models have smaller limits on token amounts.__

In [3]:
from class_version import OpenAIClient
ai = OpenAIClient(max_tokens=8100, model_name='gpt-4o-mini', system_role_content="""you are an IT administrator checking the productivity of individual employees on 
                  behalf of your manager""")
                  

### __A sample size should be entered for the second arguement of the *check_modify_df_size* function.__

### __This will be the minimum percentage of the dataset to be used in the event that the dataset is too large.__ 

In [4]:
def check_modify_df_size(df, smallest_sample_size):
    """
    Adjusts the size of a DataFrame to ensure it can be processed by a model with a token size limit.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be processed.
    smallest_sample_size (int): The smallest sample size percentage to consider when reducing the DataFrame size.

    Returns:
    str: A string representation of the DataFrame if it fits within the token size limit.
    list: A list containing an error message if the DataFrame is too large to be processed.
    """
    
    # The maximum token size that the model can process is 199,000. The number of characters per token is approximately 4.
    MAX_TOKEN_SIZE = 199000
    CHARS_PER_TOKEN = 4
    
    # List of sample sizes to be used to reduce the size of the DataFrame
    SAMPLE_SIZES = [x/100 for x in range(90, smallest_sample_size - 10, -10)]
    
    # The DataFrame must be converted to string so that the model can read it in its entirety and not just the first and 
    # last lines which is what happens in the non-string version.
    string_version = df.to_string()
    
    # Check if the string version of the DataFrame fits within the token size limit
    if len(string_version) / CHARS_PER_TOKEN <= MAX_TOKEN_SIZE:
        return string_version
    else:
        # Iterate through the sample sizes to find a reduced version of the DataFrame that fits within the token size limit
        for sample in SAMPLE_SIZES:
            sample_df = df.sample(frac=sample)
            sample_string_version = sample_df.to_string()
            if len(sample_string_version) / CHARS_PER_TOKEN <= MAX_TOKEN_SIZE:
                return sample_string_version
    
    # If no suitable sample size is found, return an error message
    if len(string_version) / CHARS_PER_TOKEN > MAX_TOKEN_SIZE:
        # the error message is returned as a list to differentiate it from the string representation of the DataFrame
        return ['The DataFrame is too large to be processed by the model.']

### __The prompt can modified in any way to get different results.__

In [5]:
# if the functions returns a DataFrame, then the DataFrame is small enough to be processed by the model
if type(check_modify_df_size(result_df, 20)) != list:
       
        ai_response = ai.get_response(f"""
        Analyze the following browser history data: {check_modify_df_size(result_df, 20)}. 
        The analysis should be on the entire dataset not just a sample. List the total number of urls visited.
        Create a table of all the unique websites visited and the number of times each website was visited.
        Identify the visited websites and categorize them into work-related/learning, and entertainment, 
        and provide the percentage of sites visited in each of these categories.
        Additionally, rate the complete browsing history  as either: highly productive, productive, neutral, unproductive, 
        or concerning. 
        """)
# if the function returns a list (the error message), then the DataFrame is too large to be processed by the model
else:
    ai_response = """ The DataFrame is too large to be processed by the model. There is a limit of roughly 800,000 characters
                      that can be processed at a time. Please reduce the size of the DataFrame by using a smaller time frame 
                      and try again. 
                  """    

# Display the AI response

display(Markdown(ai_response))


## Analysis of Browser History Data

### Total Number of URLs Visited
- Total URLs visited: **248**

### Unique Websites Visited and Visit Counts

| Website                               | Number of Visits |
|---------------------------------------|------------------|
| www.pythonanywhere.com                | 126              |
| youtube.com                           | 15               |
| chat.openai.com                       | 20               |
| google.com                            | 8                |
| coursera.org                         | 6                |
| yosefc.pythonanywhere.com            | 24               |
| help.pythonanywhere.com              | 4                |
| msn.com                               | 2                |
| login.live.com                        | 4                |
| pythonanywhere.com                    | 1                |

### Website Categories
- **Work-Related/Learning**: 
  - www.pythonanywhere.com 
  - help.pythonanywhere.com 
  - coursera.org 
  - chat.openai.com 
  - google.com (search used for programming assistance)
  
  Total count: 171 visits (PythonAnywhere: 126 + Coursera: 6 + Google for learning: 8 + Help Center: 4 + Chat: 20)

- **Entertainment**: 
  - youtube.com 
  
Total count: 15 visits

### Percentage of Each Category
- **Work-Related/Learning**: 
  \[
  \text{Percentage} = \frac{171}{248} \times 100 \approx 68.95\%
  \]
  
- **Entertainment**: 
  \[
  \text{Percentage} = \frac{15}{248} \times 100 \approx 6.05\%
  \]

### Rating of Browsing History
Given the browsing history, which mainly consists of work-related activities associated with learning, development, and programming, followed by a relatively small amount of entertainment content, I would rate the browsing history as:

**Productive.**

### Summary
- Total visited URLs: **248**
- Unique websites: **10**
- Work-Related/Learning websites: **68.95%**
- Entertainment websites: **6.05%**
- Overall rating: **Productive** 

This concludes the analysis of the browser history data.

In [34]:
profession = input("Enter your profession: ")
display(Markdown(f"### Profession: __{profession}__"))

### Profession: __computer science college instructor__

In [35]:

ai = OpenAIClient(max_tokens=1000, model_name='gpt-4o-mini', system_role_content="""you are a behavioral psychologist.""")

# Construct the prompt
prompt = f"""Here is the browsing history data for an individual who is a(n) {profession} during working hours: {check_modify_df_size(result_df, 20)}. 
Analyze the data to determine if there are any concerning patterns such as addictive behavior for someone who is a(n) {profession}."""


# Get the response from the AI
response = ai.get_response(prompt)
display(Markdown(response))


To analyze the browsing history data for the individual who is a computer science college instructor, we can look for patterns that might indicate concerning or addictive behaviors. The data primarily features visits to PythonAnywhere, a platform likely used for coding, web application development, and hosting. Here are several observations:

### 1. **Frequency and Duration of Visits**
- The individual has numerous entries logged, many of which are duplicates at short time intervals (indicating repetitive behavior). For instance:
  - Multiple visits to the same URLs (e.g., visiting the same page on PythonAnywhere repeatedly within seconds).
  - High frequency of browsing during working hours (from 10:02 AM to 10:55 AM, then again in the later hours up until 5:30 PM).

### 2. **Focus on Specific Activities**
- The browsing history shows a concentrated effort on editing files (e.g., Python scripts) and managing web apps. Activities involving coding and project development are consistent, which may be in line with the instructor's professional responsibilities.
- There are multiple visits to specific file paths and console sessions, suggesting a deep engagement with various projects or tasks.

### 3. **Potential Signs of Addictive Behavior**
- **Repetitive Patterns:** The numerous repeated visits to the same URLs within a very short span may signal compulsive checking or a need for completion.
- **Lack of Diversion:** There’s little evidence of diversifying interests; the individual seems primarily focused on specific web applications and coding tasks without breaks or shifts to other activities (leisure or social).
- **Time-Consuming Activities:** The duration spent engaged with coding-related URLs may imply an excessive commitment, potentially leading to burnout or neglect of other professional or personal responsibilities.

### 4. **Time of Browsing**
- The overall browsing pattern suggests that this individual is working long hours, with browsing activity starting early in the morning and continuing until the evening. Such prolonged periods of engagement can negatively impact both mental wellness and work-life balance.

### 5. **Context of Use**
- Considering that the individual is a college instructor, their professional duties might necessitate heavy engagement with coding and project management. However, it’s crucial to assess whether the level of engagement is healthy or if it borders on obsession where it impairs other aspects of life.

### Recommendations
To address potential concerning behaviors, the individual might consider:
- **Time Management**: Setting clear boundaries for work and personal time to prevent excessive engagement in work-related tasks during non-work hours.
- **Regular Breaks**: Incorporate scheduled breaks to engage in leisure activities, helping maintain a balance and preventing burnout.
- **Reflection on Engagement**: Periodically assess their work habits and whether the level of engagement feels sustainable over the long term. 

In conclusion, while the individual appears to be highly engaged in their professional tasks related to their role as a computer science instructor, the pattern of repetitive and lengthy browsing sessions may indicate potential concerns related to work-life balance or addictive behavior. Further self-reflection and possible behavioral adjustments may be beneficial for overall well-being.